# Chapter 5. 텍스트 생성

- 트랜스포머 기반 언어 모델은 사람이 작성한 텍스트와 거의 구분이 되지 않는 텍스트를 생성하는 매우 신기한 능력을 발휘

- 어떤 명시적인 감독(supervision)도 없이 텍스트를 생성했다는 점에서 매우 놀라움

- GPT-2와 더 강력한 후속 모델인 GPT-3는 수백만 개의 웹 페이지에서 단순히 다음 단어를 예측하는 방법을 학습해서, 다양한 종류의 입력 프롬프트를 바탕으로 글을 생성할 수 있는 광범위한 기술과 패턴 인식을 습득

- 언어 모델이 사전 훈련하는 동안 덧셈, 단어 철자 배열, 번역 같은 문맥 기반으로 다음 토큰을 예측하는 작업 시퀀스에 어떻게 노출되는지 보여줌

  - 이를 통해 얻은 지깃은 미세튜닝이나 (모델이 충분히 크다면) 추론 과정에서 효율적으로 전이(transfer)됩니다.
  
  - 이런 작업은 사전에 선택된 것이 아니며 파라미터가 수십억 개인 언어 모델을 훈련하는 대규모 말뭉치에 자연스럽게 등장

![그림5-1](image/chapter05_lm-meta-learning.png)

* 현재는 Chat GPT(GPT-3.5, GPT-4 모델로 세계를 광풍에 몰아 넣고 있음, 23년도 5월 기준)

이 장에서는 GPT-2를 사용해 언어 모델의 텍스트 생성 원리를 설명하고, 다양한 디코딩 전략이 생성된 텍스트에 미치는 영향을 살펴봄

## 5.1 일관성 있는 텍스트 생성의 어려움

지금까지 사전 훈련과 지도 학습 기반의 미세 튜닝을 조합해 NLP 문제를 다루는 데 초점을 둡니다. 시퀀스나 토큰 분류 같이 작업에 특화된 헤드에서 예측 생성은 매우 간단합니다. 일련의 로짓을 출력하고 최댓값을 선택하여 예측 클래스를 얻습니다. 또는 소프트맥스 함수를 적용해 클래스별 예측 확률을 얻습니다. 이와 달리 모델의 확률 출력을 텍스트로 변환하려면 **디코딩 방법(decoding method)** 이 필요합니다. 여기에는 텍스트 생성에만 따르는 특수한 어려움이 있습니다.

  - 디코딩은 반복적으로 수행되므로 입력이 모델의 정방향 패스를 한 번 통과할 때보다 많은 계산이 필요함
  
  - 생성된 텍스트의 품질과 다양성은 디코딩 방법과 이에 관련된 하이퍼파라미터에 따라 달라짐
  
디코딩이 어떻게 수행되는지 이해하기 위해 먼저 GPT-2의 사전 훈련 방법과 텍스트 생성에 적용하는 과정을 알아봄





다른 **자기회귀 모델**(autoregressive model) 또는 **코잘 언어 모델**(causal language model)과 마찬가지로, GPT-2는 시작 프롬프트 또는 문맥 시퀀스 $ x = x_1, x_2, ... , x_k$ 가 주어질 때 텍스트에 등장하는 토큰 시퀀스 $y = y_1, y_2, ..., y_t $의 확률 $P(y|x)$를 추정하도록 사전 훈련됨

직접 $P(y|x)$를 추정하기 위해 충분한 훈련 데이터를 획득하기란 불가능하므로, 일반적으로 확률의 연쇄 법칙(chain rule)을 사용해 조건부 확률(conditional problem)의 곱으로 나타냅니다.

$$P(y_1, ... , y_t | x) = \prod_{t=1}^{n}P(y_t | y_{<t}, x) $$

여기서 $y_{<t}$는 시퀀스 $y_1, ... , y_{t-1}$을 간략화한 식입니다. 이 조건부 확률로 자기 회귀 언어 모델링은 문장의 이전 단어가 주어지면 다음 단어를 예측한다는 직관을 얻을 수 있습니다. 앞선 식의 오른쪽 항에 위치한 확률이 이를 설명합니다. 이런 사전 훈련 목표는 과거와 미래의 문맥을 모두 사용해 마스킹된 토큰을 예측하는 BERT와 매우 다릅니다. 

이제 다음 토큰 예측 작업이 임의의 길이를 가진 텍스트 시퀀스를 생성할 때 어떻게 적용할 지 예상됩니다. 아래 그림처럼 "Transformers are the"와 같은 프롬프트로 시작하면 모델은 다음 토큰을 예측합니다. 다음 토큰이 결정되면 이를 프롬프트에 추가해 새로운 입려 시퀀스를 만들고 또 다른 토큰을 생성합니다. 이 과정을 특수한 시퀀스 종료 토큰이나 사전에 정의한 최대 길이에 도달할 때까지 반복합니다.

![그림5-3](image/chapter05_text-generation.png)

**NOTE** 출력 시퀀스가 입력 프롬프트에 따라 결정되므로 이런 종류의 텍스트 생성을 종종 **조건부 텍스트 생성(conditional text generation)** 이라 합니다. 

이 과정의 핵심은 각 타임스텝에서 어떤 토큰을 선택할지 결정하는 디코딩 방법에 있습니다. 언어 모델의 헤드는 각 스텝에서 어휘사전에 있는 토큰마다 로짓 $z_{t, i}$을 생성하므로 소프트맥스를 적용하면 가능한 다음 토큰 $w_i$에 대한 확률 분포를 얻습니다. 

$$ P(y_t = w_i | y_{<t}, x) = softmax(z_{t, i}) $$ 

대부분의 디코딩 방법은 다음과 같은 $\hat{y}$를 선택해 전체적으로 확률이 가장 높은 시퀀스를 찾습니다.

$$\hat{y} = \underset{y}{argmax}P(y|x)$$

직접 $\hat{y}$를 찾으려면 언어 모델로 가능한 모든 시퀀스를 평가해야 합니다. 이런 작업을 합리적인 시간 안에 할수 있는 알고리즘이 없으므로 근사적인 방법을 사용합니다. 이 장에서 이런 근사적인 방법 몇가지를 알아보고 고품질 텍스트를 생성하는 더 똑똑하고 복잡한 알고리즘을 점진적으로 구축하겠습니다.

## 5.2 그리디 서치 디코딩